In [92]:
from bs4 import BeautifulSoup
import requests
import re
import json

In [8]:
url = "https://take.app/seralily"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [26]:
cards = soup.find_all("div", { "data-type" : "home-card"})
cards = cards[1:]

4


In [117]:
products = []

#product cards
for card in cards:
    product = {}
    
    card_header_elements = card.find_all("a", class_=re.compile(r'HomeProductsCard_clickable__\w+'))
    
    for element in card_header_elements:
        product_name_element, product_detail_element = element.find_all("p")
        
        product_name = product_name_element.text.strip() 
        product_description = product_detail_element.text.strip()  

        product["type"] = product_name
        product["description"] = product_description
        
    # items in product
    items_list = []
    
    product_items_elements = card.find("div", {"class": "m_6d731127 mantine-Stack-root"}, recursive=False)
    product_items = product_items_elements.find_all("div", {"class": "mantine-Flex-root"})
    
    for items in product_items:
        # each 'items' contains one item info and its corresponding image
        info = items.find("div", {"class": "mantine-Stack-root"})
        if info is not None:
            
            item_name = info.find_all("p", {"class": "mantine-Text-root"})[0].text.strip()
            item_description = info.find_all("p", {"class": "mantine-Text-root"})[1].text.strip()
            item_price = info.find_all("p", {"class": "mantine-Text-root"})[2].text.strip()
            item_image = items.find("img")['src']
            
            items_list.append({
                "name": item_name,
                "price": item_price,
                "description": item_description,
                "image": item_image
            })
    
    product["items"] = items_list
    
    products.append(product)


In [118]:
output_filename = "products_data.json"

with open(output_filename, "w") as json_file:
    json.dump(products, json_file, indent=2)

print(f"Data saved to {output_filename}")

Data saved to products_data.json
